In [1]:
TEX_PATH='../example/original.tex'
MD_PATH='../example/original.md'
TRANSLATION_MD_PATH='../example/translated.md'

In [2]:
corr_list = """
Hello World!: Hello World!
"""

In [3]:
if not TEX_PATH.endswith('.tex'):
    raise ValueError('Input file path must end with .tex')
if not MD_PATH.endswith('.md'):
    raise ValueError('Output file path must end with .md')
if not TRANSLATION_MD_PATH.endswith('.md'):
    raise ValueError('Output file path must end with .md')

In [4]:
with open(TEX_PATH, 'r', encoding='utf-8') as file:
    latex = file.read()
    file.close()

In [5]:
from dotenv import load_dotenv
import sys
sys.path.append('/workspaces/latex-summarizer/src')
load_dotenv()

True

In [6]:
import nest_asyncio

from utils.latex_to_markdown import latex_to_markdown
from utils.split_text import split_md_text
from utils.translate import translate_chunks

nest_asyncio.apply()

In [7]:
md = latex_to_markdown(latex)

In [8]:
with open(MD_PATH, 'w', encoding='utf-8') as file:
    file.write(md)

In [9]:
chunks = split_md_text(md, max_chars_per_chunk=5000)

In [10]:
cnt = 0

for i in range(len(chunks)):
    chunk = chunks[i]
    if chunk.startswith('$$'):
        continue
    if chunk.startswith('|'):
        continue
    cnt += 1

print("Number of chunks need to be translated: ", cnt)

Number of chunks need to be translated:  2


In [11]:
tranlated_chunks = await translate_chunks(
    chunks, corr_list, batch_size=5, timeout=20000)

Processing batch 1 of 1


In [12]:
with open(TRANSLATION_MD_PATH, 'w', encoding='utf-8') as file:
    file.write('\n\n'.join(map(lambda chunk: str(chunk), tranlated_chunks)))
    file.close()